In [1]:
import numpy as np
import h5py
import os.path as op
import os 
import JONSWAP as J
import OmegaTheta as OT
import CreateSpectra as CS
import DtoF as DF
import bathymetry as B
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

# HOS: Initial Conditions

This script produces initial conditions in HDF5 format for the HOS model.

In [2]:
def write_hdf5(SimFolder, TestName, Nx, Ny, Lx, Ly, g, h, T, dtsave, saveflg, runsubid, Tramp, etar, phir, bathy):

    filename = SimFolder + 'initpars_' + str(TestName) + '.h5'
    
    if op.exists(filename): 
        os.remove(filename)
    
    with h5py.File(filename,'w') as file:

        file.create_dataset('Nx', data = Nx)
        file.create_dataset('Ny', data = Ny)
        file.create_dataset('Lx', data = Lx)
        file.create_dataset('Ly', data = Ly)
        file.create_dataset('g', data = g)
        file.create_dataset('KP', data = 10000)
        file.create_dataset('h', data = h)
        file.create_dataset('T', data = T)
        file.create_dataset('dtsave', data = dtsave)
        file.create_dataset('saveflg', data = saveflg)
        file.create_dataset('runsubid', data = runsubid)
        file.create_dataset('rampflg', data = 1)
        file.create_dataset('Tramp', data = Tramp)
        
    filename = SimFolder + 'initdata_' + str(TestName) + '.h5'
    
    if op.exists(filename): 
        os.remove(filename)
    
    with h5py.File(filename,'w') as file:
    
        file.create_dataset('eta0', data = etar)
        file.create_dataset('phi0', data = phir)
        file.create_dataset('bat0', data = bathy)
        
    print "Success!"
    print "Output file written."

In [3]:
SimFolder = '/home/nicole/Development/hos/python/'
TestName  = 'test'

#-- Labels for the simulation run
saveflg  = 1
runsubid = 1

#-- Wave field Parameters 
g     = 9.8               #acceleration due to gravity

#-- Discretization: Number of cells in x and y direction
alpha = 9
Nx = 2**alpha            #fourier modes in x/y directions
Ny = Nx

#kp      = 4*pi^2/g; %(Tp = 1sec)
kp      = 0.033;                #for deep water
lmbda  = 2*np.pi/kp;
omega_p = np.sqrt(g*kp);
epsilon = 0.12;
theta_p = 12;
h       = 1000;             #water depth... set high to induce deep water effects
sigma   = epsilon/2/kp;     #desired eta variance
hs      = 4*sigma;          #significant wave height (hs = 0.005)


#kpx = kp*np.cos(theta_p);
#kpy = kp*np.sin(theta_p);

#-- Length of the domain
Lx = 64*lmbda              # Lx = 1*ceil(((epsilon^-2)*pi/kp)); Lx  = 16*lambda;4.*np.pi
Ly  = Lx                   # Ly = 4.*np.pi/sqrt(3.)
     
#-- Wavenumber Domain
kx = 2*np.pi/Lx*np.arange(-Nx/2,Nx/2)
ky = 2*np.pi/Ly*np.arange(-Ny/2,Ny/2)

#-- Time parameters
Tp     = 1                     #Tp = 2*pi/omega_p;
T      = 150*Tp                #-- Final simulation time #T=100; 20; T= 2*floor((epsilon^(-2)))*Tp; #BF timescale
dtsave = 1.0

Tramp  = 200

value = "JONSWAP"

switch = {'JONSWAP': CS.create_jonswap,
          'Gauss': CS.create_gauss,
          'Stokes': CS.create_stokes}

if value in switch:
     etar, phir = switch[value](kx,ky)
else:
     pass
    #default
    
bathy = B.create_bathymetry(kx,ky, Type = 'linear')
        
plt.figure(1)
plt.contourf(kx,ky,etar, cmap=plt.get_cmap("Blues"))
plt.colorbar()
plt.title("Deviation from mean water level")
plt.xlabel("kx")
plt.ylabel("ky")
plt.show()

# Write information
write_hdf5(SimFolder, TestName, Nx, Ny, Lx, Ly, g, h, T, dtsave, saveflg, runsubid, Tramp, etar, phir, bathy)

JONSWAP.py:47: RuntimeWarning: divide by zero encountered in power
  const = np.multiply(alpha_p*g**2,np.power(omega,-5))
JONSWAP.py:53: RuntimeWarning: divide by zero encountered in power
  exponent = np.multiply(-1.25,np.power(np.divide(omega,omega_p),-4))
DtoF.py:55: ComplexWarning: Casting complex values to real discards the imaginary part
  hetaF[ikx,iky] = 0.5*norm*hetaD[ikx,iky] + 0.5*norm*np.conj(hetaD[ikxm, ikym])
DtoF.py:56: ComplexWarning: Casting complex values to real discards the imaginary part
  hphiF[ikx,iky] = -1j*0.5*norm*alpha*hetaD[ikx,iky] + 1j*0.5*alpha*norm*np.conj(hetaD[ikxm, ikym])
DtoF.py:52: RuntimeWarning: invalid value encountered in double_scalars
  norm  = np.sqrt(0.5*np.sqrt(g)*(np.sqrt(np.tanh(kh)/k**3) + h*(1/np.sqrt(k*np.tanh(kh)))*sech(kh)**2))


Success!
Output file written.


DtoF.py:52: RuntimeWarning: divide by zero encountered in double_scalars
  norm  = np.sqrt(0.5*np.sqrt(g)*(np.sqrt(np.tanh(kh)/k**3) + h*(1/np.sqrt(k*np.tanh(kh)))*sech(kh)**2))
DtoF.py:53: RuntimeWarning: divide by zero encountered in double_scalars
  alpha = np.sqrt(g/k/np.tanh(kh))
